In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools
from scipy.stats import norm

import warnings
warnings.filterwarnings("ignore")

In [21]:
df_1 = pd.read_csv('glob1_mfdexport_1week_01012010_30082023.txt',sep=';')\
    .rename(columns={'<DATE>':'DATE','<OPEN>':'OPEN','<LOW>':'LOW','<CLOSE>':'CLOSE','<VOL>':'VOL','<HIGH>':'HIGH','<TICKER>':'TICKER'})\
    #.drop(['<PER>','<TIME>','<OPENINT>'],1)
df_1.DATE = pd.to_datetime(df_1.DATE,format="%Y%m%d")
companies_1 = df_1['TICKER'].unique()

In [22]:
df_1

,TICKER,<PER>,DATE,<TIME>,OPEN,HIGH,LOW,CLOSE,VOL,<OPENINT>
0,ЛУКОЙЛ,W,2010-01-11,0,1744.00000,1771.00000,1698.60000,1731.95000,9862790,0
1,ЛУКОЙЛ,W,2010-01-18,0,1720.00000,1781.84000,1640.00000,1675.99000,11589860,0
2,ЛУКОЙЛ,W,2010-01-25,0,1656.75000,1689.50000,1598.50000,1686.73000,12051294,0
3,ЛУКОЙЛ,W,2010-02-01,0,1670.00000,1733.38000,1600.02000,1617.04000,11357620,0
4,ЛУКОЙЛ,W,2010-02-08,0,1616.24000,1634.61000,1515.00000,1517.00000,12100782,0
...,...,...,...,...,...,...,...,...,...,...
17913,Россети,W,2023-07-31,0,0.11300,0.12936,0.11154,0.12184,59093660000,0
17914,Россети,W,2023-08-07,0,0.12240,0.13528,0.12050,0.13344,31147170000,0
17915,Россети,W,2023-08-14,0,0.13462,0.13686,0.11952,0.12872,23515550000,0
17916,Россети,W,2023-08-21,0,0.13032,0.13120,0.12100,0.12564,15239500000,0


In [12]:
df_2 = pd.read_csv('glob_2mfdexport_1week_01012010_30082023.txt',sep=';')\
    .rename(columns={'<DATE>':'DATE','<OPEN>':'OPEN','<LOW>':'LOW','<CLOSE>':'CLOSE','<VOL>':'VOL','<HIGH>':'HIGH','<TICKER>':'TICKER'})\
    .drop(['<PER>','<TIME>','<OPENINT>'],1)
df_2.DATE = pd.to_datetime(df_2.DATE,format="%Y%m%d")
companies_2 = df_2['TICKER'].unique()

In [16]:
df_2

,TICKER,DATE,OPEN,HIGH,LOW,CLOSE,VOL
0,Система ао,2010-01-11,25.495,29.100,25.098,28.480,10626600
1,Система ао,2010-01-18,28.400,28.948,25.811,26.500,6685000
2,Система ао,2010-01-25,26.173,27.680,25.001,27.200,7227400
3,Система ао,2010-02-01,27.060,27.490,24.252,24.942,5775100
4,Система ао,2010-02-08,25.181,26.460,24.250,25.200,5375400
...,...,...,...,...,...,...,...
4559,FIXP-гдр,2023-07-31,424.000,438.500,389.100,417.000,3604342
4560,FIXP-гдр,2023-08-07,421.200,429.000,399.200,423.000,2333293
4561,FIXP-гдр,2023-08-14,426.000,428.400,400.100,417.700,2877139
4562,FIXP-гдр,2023-08-21,420.000,438.000,409.300,432.000,2729677


In [31]:
df = pd.concat([df_1,df_2])
companies = df['TICKER'].unique()

In [32]:
df

,TICKER,DATE,OPEN,HIGH,LOW,CLOSE,VOL
0,ЛУКОЙЛ,2010-01-11,1744.00,1771.00,1698.60,1731.95,9862790
1,ЛУКОЙЛ,2010-01-18,1720.00,1781.84,1640.00,1675.99,11589860
2,ЛУКОЙЛ,2010-01-25,1656.75,1689.50,1598.50,1686.73,12051294
3,ЛУКОЙЛ,2010-02-01,1670.00,1733.38,1600.02,1617.04,11357620
4,ЛУКОЙЛ,2010-02-08,1616.24,1634.61,1515.00,1517.00,12100782
...,...,...,...,...,...,...,...
4559,FIXP-гдр,2023-07-31,424.00,438.50,389.10,417.00,3604342
4560,FIXP-гдр,2023-08-07,421.20,429.00,399.20,423.00,2333293
4561,FIXP-гдр,2023-08-14,426.00,428.40,400.10,417.70,2877139
4562,FIXP-гдр,2023-08-21,420.00,438.00,409.30,432.00,2729677


In [33]:
full_df = pd.DataFrame(columns=['Price', 'lower','pred','upper','prob_of_growth','ticker'])

for j in companies:
    #Создаём дф и временной ряд доходностей
    company_df = df.loc[df['TICKER'] == j]
    company_df['earn'] = company_df.CLOSE.pct_change()
    series = company_df[['DATE','earn']].set_index('DATE')
    
    #Делаем train и test выборки
    historic = series.earn.iloc[:int(len(series) * 0.7)].to_list()
    test = series.earn.iloc[int(len(series) * 0.7):]
    
    
    upper_conf_int = []
    lower_conf_int = []
    predictions = []
    probabilities = []
    for i in range(len(test)):
        res = statsmodels.tsa.stattools.arma_order_select_ic(historic, ic=["aic", "bic"])
        model = ARIMA(historic, order=(res.aic_min_order[0] , 0 , res.aic_min_order[1]) )
        model_results = model.fit()
#------РАСЧЁТ ВЕРОЯТНОСТИ ПОЛОЖИТЕЛЬНОЙ ДОХОДНОСТИ------------    
        #список AR коэффициентов
        ar_params = model_results.arparams
        #делаем срез с хвоста train выборки длиной равной количеству AR коэффициентов
        ar_obs = np.array( historic[-len(ar_params):][::-1] )
        # Скалярно перемножаем их , получаем сумму произведений предыдущих значений и их коэффициентов
        if len(ar_params) != 0 :
            ar_part = np.dot(ar_params , ar_obs)
        elif len(ar_params) == 0 :
            ar_part = 0
        #список MA коэффициентов
        ma_params = model_results.maparams
        #делаем срез с хвоста train выборки длиной равной количеству MA коэффициентов
        ma_obs = np.array( model_results.resid[-len(ma_params):][::-1] )
        # Скалярно перемножаем их , получаем сумму произведений предыдущих значений и их коэффициентов
        if len(ma_params) != 0 :
            ma_part = np.dot(ma_params , ma_obs)
        elif len(ma_params) == 0 :
            ma_part = 0
        #Константа
        const = model_results.params[0]
        #Mu
        mu = ar_part+ma_part+const
        #STD
        std = np.sqrt(model_results.params[-1])
        #Сама вероятость положительной доходности в будущем месяце
        probability_of_growth = norm.cdf(mu/std, loc = 0, scale = 1)
#-----------------------------------------------------------------------    
        #ПОЛУЧЕНИЕ ДИ И ПРОГНОЗА ЦЕЛЕВОЙ ПЕРЕМЕННОЙ    
        arma_forecast = model_results.get_forecast(1)
        arma_predictions_df = arma_forecast.conf_int(alpha = 0.05)
        pred = model_results.predict(start=len(historic), end=len(historic), dynamic=False)
        predictions.append(pred[0])
        probabilities.append(probability_of_growth)
        upper_conf_int.append(arma_predictions_df[0][1])
        lower_conf_int.append(arma_predictions_df[0][0])
        historic.append(test[i])
    probabilities = pd.DataFrame(probabilities).set_index(test.index)    
    predictions = pd.DataFrame(predictions).set_index(test.index)
    upper_conf_int = pd.DataFrame(upper_conf_int).set_index(test.index)   
    lower_conf_int = pd.DataFrame(lower_conf_int).set_index(test.index)          
    #ФОРМИРОВАНИЕ ИТОГОВОГО DF 
    price_pred = pd.concat([test ,lower_conf_int, predictions , upper_conf_int , probabilities ], axis=1)
    price_pred.dropna(inplace=True)
    price_pred.columns = ['Price', 'lower','pred','upper','prob_of_growth']
    price_pred['ticker'] = j
    full_df = pd.concat([full_df,price_pred])

In [34]:
full_df.to_csv('full_df_week.csv')

In [37]:
full_df.shape

(6772, 6)